# Workout: Dataclasses & Pydantic Drills

**Rules:**
- Solve without looking at documentation
- Test validation by passing invalid data

## Setup: Install Pydantic

In [ ]:
# Run this first if pydantic not installed
# !pip install pydantic

## Dataset A: Dataclasses

### Drill A1: Basic Dataclass 🟢
**Task:** Create a Product dataclass

In [ ]:
from dataclasses import dataclass

@dataclass
class Product:
    """Product with name, price, and stock."""
    pass

# Test
laptop = Product(name="Laptop", price=999.99, stock=10)
print(laptop)  # Product(name='Laptop', price=999.99, stock=10)
print(laptop.name)  # Laptop

### Drill A2: Default Values 🟢
**Task:** Add default values and a list field

In [ ]:
from dataclasses import dataclass, field

@dataclass
class User:
    """User with name, email, and optional roles."""
    name: str
    email: str
    # Add: active (bool, default True)
    # Add: roles (list[str], default empty list) - use field!

# Test
user = User(name="Alice", email="alice@example.com")
print(user.active)  # True
print(user.roles)   # []

### Drill A3: Frozen (Immutable) Dataclass 🟡
**Task:** Create an immutable Point class

In [ ]:
from dataclasses import dataclass

# Add frozen=True
@dataclass
class Point:
    x: float
    y: float

# Test
p = Point(10, 20)

# This should raise an error
try:
    p.x = 5
except Exception as e:
    print(f"Error: {type(e).__name__}")

# Frozen dataclasses can be used as dict keys
points = {Point(0, 0): "origin", Point(1, 1): "unit"}
print(points[Point(0, 0)])  # origin

### Drill A4: Post-init Processing 🟡
**Task:** Add computed fields using __post_init__

In [ ]:
from dataclasses import dataclass, field

@dataclass
class Rectangle:
    width: float
    height: float
    area: float = field(init=False)  # Not in __init__
    
    def __post_init__(self):
        # Calculate area here
        pass

# Test
rect = Rectangle(10, 5)
print(rect.area)  # 50

## Dataset B: Pydantic Basics

### Drill B1: Basic Pydantic Model 🟢
**Task:** Create a User model with validation

In [ ]:
from pydantic import BaseModel

class User(BaseModel):
    """User with validation."""
    name: str
    age: int
    email: str

# Test valid data
user = User(name="Alice", age=30, email="alice@example.com")
print(user)

# Test coercion (string "30" becomes int 30)
user2 = User(name="Bob", age="30", email="bob@example.com")
print(type(user2.age))  # <class 'int'>

# Test validation error
try:
    User(name="Charlie", age="not a number", email="charlie@example.com")
except Exception as e:
    print(f"Validation error: {e}")

### Drill B2: Field Constraints 🟡
**Task:** Add constraints using Field

In [ ]:
from pydantic import BaseModel, Field

class Product(BaseModel):
    name: str = Field(..., min_length=1, max_length=100)
    # Add price: float, must be > 0
    # Add stock: int, must be >= 0
    # Add description: str, optional, max 500 chars

# Test
prod = Product(name="Laptop", price=999.99, stock=10)
print(prod)

# This should fail
try:
    Product(name="", price=-10, stock=5)
except Exception as e:
    print(f"Error: {e}")

### Drill B3: Custom Validators 🔴
**Task:** Add custom validation logic

In [ ]:
from pydantic import BaseModel, field_validator

class User(BaseModel):
    email: str
    password: str
    
    @field_validator("email")
    @classmethod
    def validate_email(cls, v: str) -> str:
        # Check that email contains @
        # Convert to lowercase
        pass
    
    @field_validator("password")
    @classmethod
    def validate_password(cls, v: str) -> str:
        # Check minimum length 8
        pass

# Test
user = User(email="ALICE@Example.com", password="securepassword")
print(user.email)  # alice@example.com (lowercased)

# This should fail
try:
    User(email="invalid", password="short")
except Exception as e:
    print(f"Error: {e}")

## Dataset C: Nested Models

### Drill C1: Nested Pydantic Models 🟡
**Task:** Create nested models for Order with Items

In [ ]:
from pydantic import BaseModel

class Item(BaseModel):
    product_id: str
    quantity: int
    price: float

class Order(BaseModel):
    order_id: str
    customer: str
    items: list[Item]

# Test - can pass dicts, Pydantic converts to models
order = Order(
    order_id="O001",
    customer="Alice",
    items=[
        {"product_id": "P001", "quantity": 2, "price": 29.99},
        {"product_id": "P002", "quantity": 1, "price": 99.99},
    ]
)

print(order.items[0].product_id)  # P001
print(type(order.items[0]))       # <class 'Item'>

## Dataset D: Serialization

### Drill D1: To Dict and JSON 🟢
**Task:** Serialize Pydantic models

In [ ]:
from pydantic import BaseModel

class User(BaseModel):
    name: str
    age: int
    email: str

user = User(name="Alice", age=30, email="alice@example.com")

# Convert to dict
user_dict = user.model_dump()
print(user_dict)
print(type(user_dict))  # dict

# Convert to JSON string
user_json = user.model_dump_json()
print(user_json)
print(type(user_json))  # str

### Drill D2: From Dict and JSON 🟢
**Task:** Deserialize to Pydantic models

In [ ]:
from pydantic import BaseModel

class User(BaseModel):
    name: str
    age: int
    email: str

# From dict
data = {"name": "Bob", "age": 25, "email": "bob@example.com"}
user1 = User.model_validate(data)
print(user1)

# From JSON string
json_str = '{"name": "Charlie", "age": 35, "email": "charlie@example.com"}'
user2 = User.model_validate_json(json_str)
print(user2)

## Dataset E: Real-World Pattern

### Drill E1: API Response Model 🔴
**Task:** Create models for a typical API response

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal

class APIResponse(BaseModel):
    status: Literal["success", "error"]
    message: str
    data: dict | None = None
    error_code: str | None = None

# Test success response
success = APIResponse(
    status="success",
    message="User created",
    data={"user_id": 123}
)
print(success.model_dump_json(indent=2))

# Test error response
error = APIResponse(
    status="error",
    message="Invalid email",
    error_code="VALIDATION_ERROR"
)
print(error.model_dump_json(indent=2))

### Drill E2: LLM Response Parser 🔴
**Task:** Parse structured LLM output with Pydantic

In [ ]:
from pydantic import BaseModel

class ExtractedEntity(BaseModel):
    name: str
    entity_type: str
    confidence: float

class LLMExtractionResult(BaseModel):
    entities: list[ExtractedEntity]
    summary: str

# Simulate LLM response (JSON)
llm_response = '''
{
    "entities": [
        {"name": "Apple Inc.", "entity_type": "COMPANY", "confidence": 0.95},
        {"name": "Tim Cook", "entity_type": "PERSON", "confidence": 0.92}
    ],
    "summary": "Article discusses Apple's new product launch."
}
'''

# Parse with Pydantic
result = LLMExtractionResult.model_validate_json(llm_response)
print(f"Found {len(result.entities)} entities")
for entity in result.entities:
    print(f"  - {entity.name} ({entity.entity_type}): {entity.confidence:.0%}")

## Self-Assessment

| Drill | Topic | Check |
|-------|-------|-------|
| A1-A4 | Dataclasses | ☐ |
| B1-B3 | Pydantic Basics | ☐ |
| C1 | Nested Models | ☐ |
| D1-D2 | Serialization | ☐ |
| E1-E2 | Real-World Patterns | ☐ |

**🎉 Congratulations! You've completed all Modern Python drills!**